**Task 07: Querying RDF(s)**

In [2]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

     |████████████████████████████████| 235kB 2.5MB/s 
     |████████████████████████████████| 51kB 4.2MB/s 


Leemos el fichero RDF de la forma que lo hemos venido haciendo

In [3]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")

<Graph identifier=N90ed1fd98189402187dd97cf69e20130 (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

*A) RDFLib*

In [5]:
NS= Namespace("http://somewhere#")

for s, p, o in g.triples((None, RDFS.subClassOf, NS.Person)):
  print(s)

http://somewhere#Researcher


*B) SPARQL*

In [4]:
from rdflib.plugins.sparql import prepareQuery

VCARD = Namespace("http://www.w3.org/2001/vcard-rdf/3.0#")

q1 = prepareQuery('''
  SELECT 
    ?Subclass
  WHERE { 
    ?Subclass rdf:type ns:Person. 
  }
  ''',
  initNs = {"ns": NS }
)


for r in g.query(q1):
  print(r)

(rdflib.term.URIRef('http://somewhere#SaraJones'),)
(rdflib.term.URIRef('http://somewhere#JohnSmith'),)


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


*A) RDFLib*

In [13]:
subclass = g.value(subject=None, predicate=RDF.type, object=NS.Researcher) 
print(subclass)
for s, p, o in g.triples((None, RDF.type, NS.Person)):
  print(s)

http://somewhere#JaneSmith
http://somewhere#SaraJones
http://somewhere#JohnSmith


*B) SPARQL*

In [63]:
q2 = prepareQuery('''
  SELECT 
    ?individuals
  WHERE { 
    {?individuals rdf:type ns:Person.} UNION
    {?individuals rdf:type ?x.
     ?x rdfs:subClassOf ns:Person }
  }
  ''',
  initNs = {"ns": NS }
)

for r in g.query(q2):
  print(r)

(rdflib.term.URIRef('http://somewhere#SaraJones'),)
(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#JaneSmith'),)


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


*A) RDFLib*

In [11]:
subc= g.value(None, RDFS.subClassOf, NS.Person)
subper= g.value(None, RDF.type, subc)
for s,p,o in g.triples((None, RDF.type, NS.Person)):
  for a,b,c in g.triples((s, None, None)):
    print(a,b)

for s,p,o in g.triples((subper, None, None)):
  print(s,p)

http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN
http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family
http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family
http://somewhere#JaneSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#typ

*B) SPARQL*

In [84]:
q3 = prepareQuery('''
  SELECT 
    ?individuals  ?chars
  WHERE { 
    {?individuals rdf:type ns:Person.
     ?individuals ?chars ?x.} UNION 
    {
    ?individuals rdf:type ?y.
    ?y rdfs:subClassOf ns:Person.
    ?individuals ?chars ?x.}}
  ''',
  initNs = {"ns": NS }
)

for r in g.query(q3):
  print(r)

(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given'))
(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'))
(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family'))
(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN'))
(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN'))
(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given'))
(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family'))
(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://